In [1]:
# imports
from openeo import connect, Connection
from openeo.rest.datacube import DataCube
from typing import List, Dict, Tuple, Union
import pathlib
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

%matplotlib inline

In [2]:
DEFAULT_FIGSIZE = (9, 6)

def show_tiff(filename:str, figsize=DEFAULT_FIGSIZE, vmin=None, vmax=None, rescale_percentile=97, add_colorbar=False):
    """Small helper to load a geotiff and visualize it"""
    with rasterio.open(filename) as ds:
        data = ds.read()

    fig, ax = plt.subplots(figsize=figsize)

    if len(data.shape) == 3:
        if data.max() > 500:
            p = np.percentile(data, rescale_percentile, axis=[1,2])
            data = data / p[:, None, None]
            data = np.clip(data, 0, 1)        
        data = np.moveaxis(data, 0, 2)
        if data.shape[2]<3:
            data = data[:,:,0]
        
    im = ax.imshow(data, vmin=vmin, vmax=vmax,cmap="RdYlGn")
    if add_colorbar:
        fig.colorbar(im, ax=ax, fraction=0.05)

out_dir = pathlib.Path("output")
out_dir.mkdir(parents=True, exist_ok=True)

In [3]:
# Connect to VITO backend
vito_url: str = "https://openeo.vito.be/openeo/1.0"
vito_creo_url: str = "https://openeo.creo.vito.be"
ee_url: str = "https://earthengine.openeo.org"
incd_url: str = "http://openeo1.a.incd.pt/openeo/1.1.0/"

# con: Connection = connect(ee_url).authenticate_basic(username="group1", password="test123")
# con: Connection = connect(vito_creo_url)
# con.authenticate_oidc(provider_id="egi")
con: Connection = connect(incd_url).authenticate_oidc(provider_id="egi")

To authenticate: visit https://aai.egi.eu/oidc/device and enter the user code 'Qpav6d'.
Authorized successfully.
Authenticated using device code flow.


In [4]:
cube: DataCube = con.datacube_from_process("mean", data=[2, 3, 5, 8])
cube.execute()

/opt/conda/lib/python3.9/site-packages/openeo/metadata.py:244: UserWarning: No cube:dimensions metadata
  complain("No cube:dimensions metadata")


4.5

In [5]:
con.list_collections()


[{'description': 'SENTINEL-2 is a wide-swath, high-resolution, multi-spectral imaging mission, supporting Copernicus Land Monitoring studies, including the monitoring of vegetation, soil and water cover, as well as observation of inland waterways and coastal areas. The SENTINEL-2 Multispectral Instrument (MSI) samples 13 spectral bands: four bands at 10 metres, six bands at 20 metres and three bands at 60 metres spatial resolution.',
  'extent': {'spatial': {'bbox': [[-180, -90, 180, 90]]},
   'temporal': {'interval': [['2015-07-04T00:00:00Z', None]]}},
  'id': 'SENTINEL2_L1C',
  'keywords': ['esa', 'sentinel', 'sentinel2', 's2'],
  'license': 'proprietary',
  'links': ['https://earth.esa.int/web/sentinel/user-guides/sentinel-2-msi/product-types/level-1c',
   'https://creodias.eu/data-offer'],
  'providers': [],
  'stac_extensions': ['datacube'],
  'stac_version': '0.9.0',
  'title': 'Sentinel-2 L1C'},
 {'description': 'SENTINEL-2 is a wide-swath, high-resolution, multi-spectral imaging mission, supporting Copernicus Land Monitoring studies, including the monitoring of vegetation, soil and water cover, as well as observation of inland waterways and coastal areas. The SENTINEL-2 Multispectral Instrument (MSI) samples 13 spectral bands: four bands at 10 metres, six bands at 20 metres and three bands at 60 metres spatial resolution.\n The level 2A data is atmospherically corrected using Sen2Cor.',
  'extent': {'spatial': {'bbox': [[-180, -90, 180, 90]]},
   'temporal': {'interval': [['2015-07-04T00:00:00Z', None]]}},
  'id': 'SENTINEL2_L2A',
  'keywords': ['esa', 'sentinel', 'sentinel2', 's2'],
  'license': 'proprietary',
  'links': ['https://earth.esa.int/web/sentinel/user-guides/sentinel-2-msi/product-types/level-2a',
   'https://creodias.eu/data-offer'],
  'providers': [],
  'stac_extensions': ['datacube'],
  'stac_version': '0.9.0',
  'title': 'Sentinel-2 L2A'},
 {'description': 'This Vegetation Indices product includes the Normalized Difference Vegetation Index (NDVI), the normalized differece of near inrared and red reflectances, the Plant Phenology Index, the Fraction of Absorbed Photosynthetically Active Radiation (FAPAR) and the Leaf Area Index (LAI). These indices are derived from the Copernicus Sentinel-2 satellite observations and updated daily.',
  'extent': {'spatial': {'bbox': [[-25, 27, 45, 72]]},
   'temporal': {'interval': [['2017-01-01T00:00:00Z',
      '2023-12-31T00:00:00Z']]}},
  'id': 'EEA_VEGETATION_INDICES',
  'keywords': ['NDVI',
   'LAI',
   'FAPAR',
   'PPI',
   'index',
   'land',
   'vegetation',
   'plant',
   'phenology',
   'growing season',
   'productivity',
   'sentinel'],
  'license': 'proprietary',
  'links': [{'href': 'https://land.copernicus.eu/pan-european/biophysical-parameters/high-resolution-vegetation-phenology-and-productivity',
    'rel': 'alternate'}],
  'stac_extensions': ['datacube'],
  'stac_version': '0.9.0',
  'title': 'Sentinel-2 derived vegetation indices (NDVI, FAPAR, PPI and LAI) at 10m resolution over European mainland. These indices are provided by the High Resolution Vegetation Phenology and Productivity service, part of the Copernicus land monitoring service.'},
 {'description': 'The WorldCover product will be released per 3 x 3 degree tile.',
  'extent': {'spatial': {'bbox': [[-180.0, -90.0, 180.0, 90.0]]},
   'temporal': {'interval': [['2020-01-01T00:00:00Z',
      '2020-12-31T00:00:00Z']]}},
  'id': 'ESA_WORLDCOVER_10M_2020_V1',
  'keywords': ['VITO',
   'COPERNICUS',
   'ESA',
   'Land Cover',
   'Sentinel-1',
   'Sentinel-2',
   'Environment',
   'WorldCover'],
  'license': 'https://creativecommons.org/licenses/by/4.0/',
  'links': [{'href': 'https://creativecommons.org/licenses/by/4.0/',
    'rel': 'license'},
   {'href': 'https://services.terrascope.be/catalogue/description?collection=urn:eop:VITO:ESA_WorldCover_10m_2020_V1',
    'rel': 'alternate',
    'title': 'EO OpenSearch catalog'},
   {'href': 'https://viewer.esa-worldcover.org/worldcover',
    'rel': 'alte

In [7]:
bbox = {
    'west': 5.017099,
    'south': 51.219585,
    'east': 5.023468,
    'north': 51.223363,
    'crs': 'EPSG:4326',
}

s2: DataCube = con.load_collection("SENTINEL2_L1C",
    spatial_extent=bbox,
    temporal_extent=["2019-01-01", "2019-04-01"],
    bands=["B02", "B03", "B04"]).rename_labels(dimension="bands", target=["R", "G", "B"], source=["B04", "B03", "B02"]
)

# s2 = s2.reduce_dimension(dimension="t", reducer="median")

s2.download(out_dir / "s2.tiff", format="GTiff")
# s2.download(out_dir / "s2.json", format="JSON")
# s2.download(out_dir / "s2.png", format="png")

s2.graph
    

OpenEoApiError: [500] Internal: Server error: Unable to construct dataset dimensions. GDAL Error Code: 4 (ref: r-9332f0e3c3034df0b9624e25ebcc9fba)

In [ ]:
import rasterio

dataset: rasterio.DatasetReader = rasterio.open(out_dir / "s2.tiff")
plt.imshow(dataset.read(1))
plt.show()